In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from keras.datasets import mnist
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Flatten
#from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, callbacks

In [2]:
dataset_url = '../screen_scam/photo_all'

In [22]:
seed_train_validation = 1 # Must be same for train_ds and val_ds
shuffle_value = True
validation_split = 0.3

train_ds = tf.keras.utils.image_dataset_from_directory(
directory =dataset_url,
validation_split = validation_split,
subset = "training",
seed = seed_train_validation,
shuffle = shuffle_value)

val_ds = tf.keras.utils.image_dataset_from_directory(
directory =dataset_url,
validation_split = validation_split,
subset = "validation",
seed = seed_train_validation,
shuffle = shuffle_value)

Found 4259 files belonging to 2 classes.
Using 2982 files for training.
Found 4259 files belonging to 2 classes.
Using 1277 files for validation.


In [23]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take((2*val_batches) // 3)
val_ds = val_ds.skip((2*val_batches) // 3)

In [26]:
from tensorflow.keras import optimizers

In [27]:
def load_own_model():
    # Model Architecture
    model = Sequential([
        # Conv + Pool
        layers.Rescaling(scale=1./255, input_shape=(256,256,3)),
        layers.Conv2D(32, (8, 8), activation='relu'),
        layers.MaxPool2D(3, 3),
        layers.Conv2D(64, (6, 6), activation='relu'),
        layers.MaxPool2D(3, 3),
        layers.Conv2D(64, (4, 4), activation='relu'),
        layers.MaxPool2D(3, 3),
        # Flatten + Dense
        layers.Flatten(),
        layers.Dense(16, activation='relu'),
        # Output
        layers.Dense(1, activation='sigmoid')
    ])
    
    # Compile
    adam_opt = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    return model

In [28]:
from tensorflow.keras import callbacks



In [30]:

model = load_own_model()

batch_size = 256
epochs = 50
es = callbacks.EarlyStopping(patience=30, restore_best_weights=True)

history = model.fit(train_ds,
                    batch_size=32,
                    epochs=epochs,
                    validation_data=val_ds,
                    callbacks=[es])



Epoch 1/50
94/94 [==============================] - 24s 232ms/step - loss: 0.6953 - accuracy: 0.5181 - val_loss: 0.6922 - val_accuracy: 0.5236
Epoch 2/50
94/94 [==============================] - 22s 223ms/step - loss: 0.6823 - accuracy: 0.5503 - val_loss: 0.6598 - val_accuracy: 0.6000
Epoch 3/50
94/94 [==============================] - 23s 226ms/step - loss: 0.6618 - accuracy: 0.5929 - val_loss: 0.6546 - val_accuracy: 0.6112
Epoch 4/50
94/94 [==============================] - 22s 222ms/step - loss: 0.6418 - accuracy: 0.6244 - val_loss: 0.6557 - val_accuracy: 0.6157
Epoch 5/50
94/94 [==============================] - 22s 223ms/step - loss: 0.6276 - accuracy: 0.6429 - val_loss: 0.6839 - val_accuracy: 0.5933
Epoch 6/50
94/94 [==============================] - 22s 224ms/step - loss: 0.6111 - accuracy: 0.6539 - val_loss: 0.6464 - val_accuracy: 0.6787
Epoch 7/50
94/94 [==============================] - 22s 224ms/step - loss: 0.5827 - accuracy: 0.6804 - val_loss: 0.6771 - val_accuracy: 0.6270

In [31]:
model.evaluate(test_ds)

26/26 [==============================] - 5s 155ms/step - loss: 0.6304 - accuracy: 0.6707


[0.630433201789856, 0.6706730723381042]

<bound method _NumpyIterator.next of <tensorflow.python.data.ops.dataset_ops._NumpyIterator object at 0x7f02665d6290>>